In [1]:
import os

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import Simulator, StartMeshcat

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
def make_stack():
    output = ''
    num_layers = 18
    num_blocks_per_layer = 3
    block_width = 0.025
    block_height = 0.015
    for i in range(num_layers):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(num_blocks_per_layer):
            x = (block_width * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (block_width * (j - 1))
            z = block_height * i
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            output += f"""
            - add_model:
                name: block_{i * 3 + j}
                file: /models/jenga_block.sdf
                default_free_body_pose:
                    jenga_block_link:
                        translation: [{x}, {y}, {z}]
                        rotation: !Rpy {orientation}
            """
    return output

make_stack()

'\n            - add_model:\n                name: block_0\n                file: /models/jenga_block.sdf\n                default_free_body_pose:\n                    jenga_block_link:\n                        translation: [-0.025, 0, 0.0]\n                        rotation: !Rpy { deg: [0, 0, 0] }\n            \n            - add_model:\n                name: block_1\n                file: /models/jenga_block.sdf\n                default_free_body_pose:\n                    jenga_block_link:\n                        translation: [0.0, 0, 0.0]\n                        rotation: !Rpy { deg: [0, 0, 0] }\n            \n            - add_model:\n                name: block_2\n                file: /models/jenga_block.sdf\n                default_free_body_pose:\n                    jenga_block_link:\n                        translation: [0.025, 0, 0.0]\n                        rotation: !Rpy { deg: [0, 0, 0] }\n            \n            - add_model:\n                name: block_3\n        

In [4]:
def create_scene():
    scenario_data = f"""
    directives:
    - add_model:
        name: table_top
        file: /models/table_top.sdf
    - add_weld:
        parent: world
        child: table_top::table_top_center
    """

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()

create_scene()

RuntimeError: error: URI '/models/table_top.sdf' is invalid when parsing a string instead of a filename.